In [42]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama  # Solution locale
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import gradio as gr
import torch
import warnings
warnings.filterwarnings('ignore')
import tqdm as notebook_tqdm
from langchain.prompts import PromptTemplate

In [23]:

pdf_files = ["cv_fr.pdf", "Valentin_Kocijancic_CV_2025 .pdf"]

# chargement pdf
documents = []
for file in pdf_files:
    loader = PyPDFLoader(file)
    docs = loader.load()
    documents.extend(docs)

# chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # taille des chunks
    chunk_overlap=200,    # chevauchement pour garder le contexte
    length_function=len
)
chunks = text_splitter.split_documents(documents)

print(f"Nombre de chunks créés : {len(chunks)}")

Nombre de chunks créés : 6


In [24]:
# 1. Choisir le modèle d'embedding
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'}  # 'cuda' si GPU
)

# 2. Créer la base vectorielle FAISS
vector_store = FAISS.from_documents(chunks, embedding_model)

# 3. Sauvegarder la base
vector_store.save_local("faiss_index")
#vector_store = FAISS.load_local("faiss_index", embedding_model,allow_dangerous_deserialization=True)

#print("Base FAISS créée et sauvegardée !")

In [25]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [43]:
llm = Ollama(
    model="phi3",
    temperature=0.1,  # Pour des réponses plus factuelles
    num_predict=256   # Limite la longueur des réponses
)

In [44]:
template = """
Tu es un assistant intelligent spécialisé dans l'analyse de CV pour les recruteurs.
Tu dois répondre **en français** de manière claire, concise et professionnelle.

Tu disposes des informations suivantes issues d'une base de CV :
{context}

Consignes :
- Utilise uniquement les informations présentes dans le contexte pour répondre.
- Si la réponse n'est pas clairement indiquée, dis simplement : "L'information n'est pas disponible dans les CV."
- Si plusieurs candidats semblent correspondre, mentionne leurs prénoms et explique brièvement pourquoi.
- Si un candidat se démarque particulièrement, indique-le clairement et justifie ton choix.
- Ne traduis pas les noms de postes ou d’entreprises.

Question du recruteur : {question}

Réponse :
"""

prompt_fr = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [45]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_fr}
)

In [29]:
question = "Quel candidat possède la meilleur expérience pour le développement d'algorithmes ?"
response = qa_chain({"query" : question}, return_only_outputs=False)
print("Réponse :", response["result"])
print("Source :", response["source_documents"][0].page_content)

Réponse :  Le candidat Axel Bröns possède une expérience remarquable dans le développement d'algorithmes. Il a effectué un stage en algorithmique et modélisation quantitative à L’Immobiliere De Réseau où il a conçu, implémenté et testé environ 20 stratégies de trading algorithmique sur cTrader en C#. Il a également optimisé les hyperparamètres des stratégies via cAlgo et PineScript, visant un facteur de profit supérieur à 2,5. En outre, il a modélisé mathématiquement le risque et le rendement à l'aide du critère de Kelly. Ces expériences démontrent une solide compétence en développement d'algorithmes.
Source : langages (RoBERTa, BERT) (en cours)
A PROPOS DE MOI
Étudiant (bac +4) en école d’ingénieurs,
passionné par l’intelligence artiﬁcielle, la data
et la recherche. Curieux et rigoureux, je
recherche un stage de 4 mois minimum à
partir du 13 avril 2026 à Paris ou Lyon.
FORMATIONS
Diplôme d’ingénieurs
École centrale d’électronique - ECE
Ὄ52022 – 2027 ♂¶ap-¶arkerParis, France
• Majeure 

In [46]:
question = "Quel candidat cherche un stage en finance ?"
response = qa_chain({"query" : question})
print(response['result'])

Valentin Kocijančić est le candidat qui recherche un stage en finance, comme indiqué dans son CV où il mentionne explicitement qu'il "souhaite appliquer mes compétences techniques et mon esprit analytique au sein d’une équipe dynamique et stimulante." Il cherche également à commencer le stage dès le 13 avril 2026, ce qui correspond aux dates mentionnées dans la question du recruteur.
